In this new version of the testing space I will be tweaking at the response function I created in order to generate some sort of look up table which will be used to help design the response function for each detector, as well as the chi squared response! 

Original Functions
==========

In [1]:

def chiresponse(A):
    """
    The response function used in the chi squared fitting portion of the simulation. 
    Meant to imitate the actual response of a scintillator.
    Inputs 2 vectors, and responds with a cos^x dependence.
    
    Parameters
    ----------
    A : float
        The angle between the two vectors who's response is meant to be imitated. 

    Returns
    -------

    A : float
        The cosine dependence based on the angle, includes a mask so that terms corresponding to angular separations beyond pi/2 are 0, imitating what would happen if a GRB didn't strike the face of a detector. Further simulations of this effect are neccessary in a different software package to confirm this assumption, but its okay for now. 

    """
    #meant to imitate the response of the detectors for effective area vs. angle, found to be around .77
 #   print(length(A),length(B))
#if cosine is negative, 

    mask = A > np.pi/2.

    A[mask] = 0
    A[~mask] = pow(abs(np.cos(A[~mask])),0.76)
    
    
    return A

def response(A):
    """
    Meant to imitate the actual response of a scintillator.
    Inputs 2 vectors, and responds with a cos^x dependence.
    
    Parameters
    -----------
    A : float
        The angular separation in radians between the normal vector of the detector, and the position in the sky of the simulated GRB. 

    Returns
    -------
    R : float
        The response function of how the scintillator will respond to a source at angle A. 

    """
    #meant to imitate the response of the detectors for effective area vs. angle, found to be around .77
 #   print(length(A),length(B))
#if cosine is negative, 

    R = pow(abs(np.cos(A)),0.76)
    
    
    return R         



First of all, there's no real reason for there to be two of these in the first place. Response should be replaced by the "chiresponse" version in all places in the code,and just for simplicity rename that response. 

In [ ]:
def response(A):
    """
    Meant to imitate the actual response of a scintillator.
    Inputs 2 vectors, and responds with a cos^x dependence.
    
    Parameters
    ----------
    A : float
        The angle between the two vectors who's response is meant to be imitated. 

    Returns
    -------

    A : float
        The cosine dependence based on the angle, includes a mask so that terms corresponding to angular separations beyond pi/2 are 0, imitating what would happen if a GRB didn't strike the face of a detector. Further simulations of this effect are neccessary in a different software package to confirm this assumption, but its okay for now. 

    """
    mask = A > np.pi/2.

    A[mask] = 0
    A[~mask] = pow(abs(np.cos(A[~mask])),0.76)
    
    
    return A

Now I want to make a lookup table for each detector, note each has a different normal, and in these different domains have varying response functions. Let's do a dumb one now.

In [21]:
import numpy as np
def response(detnorm,skyvec):
    """
    Meant to imitate the actual response of a scintillator.
    Inputs 2 vectors, and responds with a cos^x dependence.
    
    Parameters
    ----------
    detnorm : array
        The normal vector of the detector. 
    skyvec : array
        The position of GRB in xyz coord. 

    Returns
    -------

    A : float
        The cosine dependence based on the angle, includes a mask so that terms corresponding to angular separations beyond pi/2 are 0, imitating what would happen if a GRB didn't strike the face of a detector. Further simulations of this effect are neccessary in a different software package to confirm this assumption, but its okay for now. 

    """
    from burstutils import findAngles   
    from healpy import ang2vec,vec2ang 
    theta,phi = vec2ang(skyvec)
    A = findAngles(detnorm,skyvec)

    if theta[0]  < np.pi/8:
        mask = A > np.pi/2

        A[mask] = 0
        A[~mask] = pow(abs(np.cos(A[~mask])),1)
    else:
        mask = A > np.pi/2

        A[mask] = 0
        A[~mask] = pow(abs(np.cos(A[~mask])),.75)
    

    
        
    return A

This appears to be a VERY simple version of this code which changes the response function based on where the theta and phi are of the GRB. Is this right? 